In [412]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [413]:
from sklearn import linear_model
from helper.plot_pr_roc import plot_pr_curve, plot_roc_curve
import warnings
warnings.filterwarnings("ignore")
from config.paths import TRAIN_FILE
from config.paths import OUT_FIG_DIR, OUT_DATA_DIR
from sklearn.metrics import roc_curve, auc
from lib.helper import prepare_data
from lib.agent import Agent

In [414]:
df = pd.read_csv(TRAIN_FILE).drop(columns=['Gene_name'])
df.iloc[:, :7] = df.iloc[:, :7] - df.iloc[:, :7].min()
df.iloc[:, :7]  /= df.iloc[:, :7].max()

In [415]:
df_train = df[df.Category=="Training set"].drop(columns=["Category"])
df_test = df[df.Category!="Training set"].drop(columns=["Category"])

df_train, df_test = prepare_data(TRAIN_FILE)

In [416]:
agent = Agent(df_train, df_test, costs=[1, 5, 10, 15, 100, 100], bootstrapping=False)
agent.run()

In [417]:
y_hat_pycharm = agent.y_hat_test

In [418]:
agent2 = Agent(df_train, df_test, costs=[1, 5, 10, 15, 100, 100], bootstrapping=False)
agent2.run()

In [419]:
df["prediction"] = np.concatenate([agent.y_hat_train, agent.y_hat_test])

In [420]:
res_path = OUT_DATA_DIR + "/prediction.csv"
df.to_csv(res_path)

In [421]:
pred = pd.DataFrame({'pre': agent.y_hat_test, 'truth': agent.y_test})

In [422]:
pred.sort_values(by="pre", inplace=True)

In [423]:
def agent_one_dataset(df_train, df_test, feature="All feature"):
    agent = Agent(df_train, df_test, costs=[1, 5, 10, 15, 100, 100])
    agent.run()
    
    which_parts = ["train", "test"]
    for which_part in which_parts:
        save_dir = OUT_FIG_DIR + "/%s" % which_part
        os.makedirs(save_dir, exist_ok=True)
        save_path = save_dir + "/%s_roc.pdf" % feature
        if which_part == "test":
            auc_roc_test = plot_roc_curve(agent.y_test, agent.y_hat_test,
                                          save_path, feature)
        else:
            auc_roc_train = plot_roc_curve(agent.y_train, agent.y_hat_train,
                                           save_path, feature)
            
    for which_part in which_parts:
        save_dir = OUT_FIG_DIR + "/%s" % which_part
        os.makedirs(save_dir, exist_ok=True)
        save_path = save_dir + "/%s_pr.pdf" % feature
        if which_part == "test":
            auc_pr_test = plot_pr_curve(agent.y_test, agent.y_hat_test,
                                        save_path, feature)
        else:
            auc_pr_train = plot_pr_curve(agent.y_train, agent.y_hat_train,
                                        save_path, feature)
            
    return auc_roc_train, auc_pr_train, auc_roc_test, auc_pr_test

In [424]:
aucs = []
features = ["all_features"]
results = agent_one_dataset(df_train, df_test, feature="")
aucs.append(results)